# Viewing results in Napari

Code uses outputs created by the spot_detection script to quickly load a particular image with masks and spots into napari in order to visually evaluate the restults.

Run this on a local machine rather than the server as getting X11 forwarding to work on the server is a bit tricky. 

Pathnames from the fileList.csv df are automatically converted from /mnt/external.data/ to /Volumes/external.data/

If you are working on windows, then mount izbkingston in WSL under /mnt/izbkingston/ so it can be converted to /mnt/external.data/.

the pathnames stored in the file_list.csv of flie_list_wormMasks.csv are always in server format.

In [ ]:
import napari
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from bioio import BioImage
import bioio_nd2
import tqdm
from bioio.writers import OmeTiffWriter
from magicgui import magicgui
from napari.settings import get_settings
from magicgui.widgets import ComboBox, PushButton, Container
from skimage.draw import polygon
from convert_paths import correct_path, correct_save_path, correct_loaded_path


settings = get_settings()
settings.application.ipy_interactive = False


## Input parameters

In [ ]:
spotChannel = 0 # green spots
#nucChannel = 0 # red emerin rings  - if there is no nuclear marker, set to same as spotChannel

path_type="mac" # could be "server", "mac" or "wsl"

#output_path = '/mnt/external.data/MeisterLab/jsemple/lhinder/segmentation_Dario/DPY27/'
output_path = '/mnt/external.data/MeisterLab/jsemple/demo_VIBE/results/2025-10-05_bet1-mSG_wPM1353'
output_path = correct_path(output_path, path_type)
 
df = pd.read_csv(os.path.join(output_path,'fileList.csv'))
df = correct_loaded_path(df, path_type, col_names=['raw_filepath', 'denoised_filepath'])

df.head()

,filename,date,protein,strain,treatment,worm_id,id,raw_filepath,denoised_filepath
0,2025_10_05_wPM1353_HS_001,2025-10-05,bet1-mSG,wPM1353,HS,1,bet1-mSG_2025-10-05_2025_10_05_wPM1353_HS_001,/Volumes/external.data/MeisterLab/jsemple/demo...,/Volumes/external.data/MeisterLab/jsemple/demo...
1,2025_10_05_wPM1353_nHS_001,2025-10-05,bet1-mSG,wPM1353,nHS,1,bet1-mSG_2025-10-05_2025_10_05_wPM1353_nHS_001,/Volumes/external.data/MeisterLab/jsemple/demo...,/Volumes/external.data/MeisterLab/jsemple/demo...


## Function to interactively choose and clasify region

In [ ]:
def select_worm_region(df, index, output_path, spotChannel=0, contrast_min=100, contrast_max=250):
    img = BioImage(df.raw_filepath.iloc[index], reader=bioio_nd2.Reader)
    img = img.get_image_data("ZYX", T=0, C=spotChannel)
    img = np.max(img[:, :, :], axis=0)

    if os.path.exists(os.path.join(output_path,'worm_masks'))==False:
        os.makedirs(os.path.join(output_path,'worm_masks'), exist_ok=True)

    if os.path.exists(os.path.join(output_path,'qc'))==False:
        os.makedirs(os.path.join(output_path,'qc'), exist_ok=True)

    # Create the Napari viewer
    viewer = napari.Viewer()

    # Add the image and shapes layers
    layer_c0 = viewer.add_image(img, blending='additive', colormap='cyan', opacity=0.85,
                            contrast_limits=[contrast_min, contrast_max])

    shapes_layer = viewer.add_shapes(opacity=0.5, name='Labeled Shapes', edge_color='white', 
                                face_color='transparent', edge_width=5, properties={'label': []},
                                text={'text': 'label', 'size': 20, 'color': 'magenta', 'anchor': 'center'})


    # Dropdown for label choices (no "other" option)
    label_dropdown = ComboBox(label="Label", choices=["head", "tail", "tailnuc", "body_vnc","body_other","embryo","worm"])

    # Button to apply the label
    set_label_button = PushButton(label="Set Label")

    # Function to apply the label to the current shape
    def apply_label(event=None):
        label = label_dropdown.value
        shapes_layer.current_properties = {'label': [label]}

    set_label_button.clicked.connect(apply_label)

    # Combine widgets into a container
    label_widget = Container(widgets=[label_dropdown, set_label_button])

    # Add to Napari
    viewer.window.add_dock_widget(label_widget, area='right')

    # Add a text prompt to the viewer
    viewer.text_overlay.text = "Draw polygon outline in 'labled Shapes' layer then press 'm' to save the mask."
    viewer.text_overlay.visible = True

    # Keybinding to save shapes with the same label as binary masks
    @viewer.bind_key('m')
    def save_shapes_as_masks(viewer):
        shapes = shapes_layer.data
        labels = shapes_layer.properties['label']
        canvas_size = img.shape  # Adjust to your image size

        for label in set(labels):
            mask = np.zeros(canvas_size, dtype=np.uint8)
            for shape, shape_label in zip(shapes, labels):
                if shape_label == label:
                    rr, cc = polygon(shape[:, 0], shape[:, 1], canvas_size)
                    mask[rr, cc] = 1
            mask_path = os.path.join(output_path, 'worm_masks', df.id.iloc[index] + '_' + label +'.tif')
            OmeTiffWriter.save(mask.astype(np.uint8), mask_path)

        if 'worm_regions_all' not in df.columns:
            df['worm_regions_all'] = ""
        df.at[index, 'worm_regions_all'] = ';'.join(set(labels))
        viewer.text_overlay.text = 'index'+str(index)+' '+df.at[index, 'worm_regions_all']
        viewer.text_overlay.font_size = 40
        viewer.text_overlay.visible = True
        viewer.export_figure(os.path.join(output_path, 'qc', 'wormMask_'+df.id.iloc[index]  + '.png'))

        viewer.close()
    napari.run()
    return df


## Manually create masks for worm region
Run this code to scroll through the rows of the df dataframe, and open a max projection of the raw image in napari. To create mask:

1) Select from dropdown menu on the top right what worm region you are highlighting (head, tail, tailnuc, body, embryo), and press "select" button

2) Use polygon selection tool among the options on the left menu to draw an outline around the region of interest. It does not have to be tight around the worm, just avoid any other fluorescent regions you are not interestd in. If you make a mistake you can delete and redraw. you can draw several regions of the same type.

3) To draw a region of a different type, select a different region from the dropdown menu and press the "set label" button.

4) When you are satisfied with your selection, press 'm' and that will save a binary mask for each type of region, and save the region types you selected to the data frame and close the napari viewer.


The code saves the df dataframe into fileList_wormMasks.csv after every image, so if your run get interrupted you can just change the range of indeces you are using.

In [13]:
if os.path.exists(os.path.join(output_path,'fileList_wormMasks.csv'))==True:
    df = pd.read_csv(os.path.join(output_path,'fileList_wormMasks.csv'))
    df = correct_loaded_path(df, path_type, col_names=['raw_filepath', 'denoised_filepath'])


indeces=range(0, len(df))
#indeces=range(19, 20)

for index in tqdm.tqdm(indeces):
    df = select_worm_region(df, index, output_path, spotChannel=spotChannel, contrast_min=100,contrast_max=450)
    df_tmp = df.copy()
    df_tmp = correct_save_path(df_tmp, path_type, col_names=['raw_filepath', 'denoised_filepath'])
    df_tmp.to_csv(os.path.join(output_path,'fileList_wormMasks.csv'),index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
 50%|█████     | 1/2 [01:32<01:32, 92.03s/it]/Users/semple/micromamba/envs/lhcellpose/lib/python3.10/site-packages/dask/tokenize.py:242: UserWarning: ND2File file not closed before garbage collection. Please use `with ND2File(...):` context or call `.close()`.
  mod.loads(out, buffers=buffers)
100%|██████████| 2/2 [02:48<00:00, 84.39s/it]
